In [62]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline  

### Open the data for human and see what we have.

In [63]:
human = pd.read_table('dna.txt')
human.shape

(22738, 2)

### We have some data for human DNA sequence coding regions and a class label. We also have data for Chimpanzee and a more divergent species, the dog. Let's get that.

In [64]:
human.shape


(22738, 2)

In [65]:
# function to convert sequence strings into k-mer words, default size = 6 (hexamer words)
def getKmers(sequence, size=6):
    return [sequence[x:x+size].lower() for x in range(len(sequence) - size + 1)]

### Now we can convert our training data sequences into short overlapping k-mers of legth 6. Lets do that for each species of data we have using our getKmers function.

In [66]:
human["words"]=human.apply(lambda x: getKmers(x["sequence"]),axis=1)
human.head()
#chimp['words'] = chimp.apply(lambda x: getKmers(x['sequence']), axis=1)
#chimp = chimp.drop('sequence', axis=1)
#dog['words'] = dog.apply(lambda x: getKmers(x['sequence']), axis=1)
#dog = dog.drop('sequence', axis=1)

,sequence,1,words
0,TGCGAGGGATTCTTCTATACCGCCGGCACTTATTTTTTTGCCCACG...,1,"[tgcgag, gcgagg, cgaggg, gaggga, agggat, gggat..."
1,CGCGGCGCTCATGTGTTACGATTCCTTCCAGGTTATCGACCGACAT...,1,"[cgcggc, gcggcg, cggcgc, ggcgct, gcgctc, cgctc..."
2,TTGCGCCAGTTGCGGCAGGCTGTCAGGGAACAGGGGGTACGGGATG...,1,"[ttgcgc, tgcgcc, gcgcca, cgccag, gccagt, ccagt..."
3,AACTCGGTCATAGGGGGATGTCGGCCGACAGGCTGGAGCTACTGGG...,1,"[aactcg, actcgg, ctcggt, tcggtc, cggtca, ggtca..."
4,CAGAAGCCCTGAATCGGTCGTTAGGCTGGCAGTTGCTAAAGAGCCG...,1,"[cagaag, agaagc, gaagcc, aagccc, agccct, gccct..."


In [67]:
human=human.drop("sequence",axis=1)
#human["words"][0]
human.head()


,1,words
0,1,"[tgcgag, gcgagg, cgaggg, gaggga, agggat, gggat..."
1,1,"[cgcggc, gcggcg, cggcgc, ggcgct, gcgctc, cgctc..."
2,1,"[ttgcgc, tgcgcc, gcgcca, cgccag, gccagt, ccagt..."
3,1,"[aactcg, actcgg, ctcggt, tcggtc, cggtca, ggtca..."
4,1,"[cagaag, agaagc, gaagcc, aagccc, agccct, gccct..."


In [68]:
#human["words"][2]


In [69]:
human_texts=list(human["words"])
for item in range(len(human_texts)):
    human_texts[item] = ' '.join(human_texts[item])
y_h = human.iloc[:, 0].values  

In [70]:
human_texts[2]

'ttgcgc tgcgcc gcgcca cgccag gccagt ccagtt cagttg agttgc gttgcg ttgcgg tgcggc gcggca cggcag ggcagg gcaggc caggct aggctg ggctgt gctgtc ctgtca tgtcag gtcagg tcaggg caggga agggaa gggaac ggaaca gaacag aacagg acaggg cagggg aggggg gggggt ggggta gggtac ggtacg gtacgg tacggg acggga cgggat gggatg ggatga gatgat atgata tgatag gatagt atagtt tagtta agttaa gttaaa ttaaat taaatt aaattt aatttc atttca tttcat ttcatg tcatgc catgcc atgcca tgccag gccagg ccaggg cagggg aggggg ggggga ggggaa gggaaa ggaaag gaaagg aaaggc aaggcg aggcgg ggcggt gcggtg cggtgg ggtgga gtggaa tggaag ggaagt gaagtg aagtgg agtggc gtggcc tggccc ggcccg gcccgg cccggt ccggtc cggtcg ggtcga gtcgat tcgatt cgatta gattac attacc ttacct tacctg acctgc cctgct ctgctg tgctgg gctggg ctgggg tggggc ggggcg gggcgg ggcggg gcggga cgggag gggagc ggagcg gagcgt agcgta gcgtaa cgtaac gtaacc taaccg aaccgc accgcg ccgcga cgcgaa gcgaag cgaagg gaaggc aaggcg aggcgc ggcgca gcgcaa cgcaac gcaacg caacgg aacggt acggtg cggtgc ggtgct gtgctc tgctcc gctccg ctccgg tccggg ccgggg cgggg

In [71]:
y_h

array([1, 1, 1, ..., 0, 0, 0], dtype=int64)

## Now let's review how to use sklearn's "Natural Language" Processing tools to convert our k-mer words into uniform length numerical vectors that represent counts for every k-mer in the vocabulary.

In [72]:
# Creating the Bag of Words model using CountVectorizer()
# This is equivalent to k-mer counting
# The n-gram size of 4 was previously determined by testing
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(4,4))
X = cv.fit_transform(human_texts)
#X_chimp = cv.transform(chimp_texts)
#X_dog = cv.transform(dog_texts)

## Let's see what we have... for human we have 4380 genes converted into uniform length feature vectors of 4-gram k-mer (length 6) counts.

In [74]:
print(X.shape)


(22738, 260449)


In [75]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,y_h,test_size=0.2,random_state=42)

In [76]:
print(x_train.shape)
print(x_test.shape)

(18190, 260449)
(4548, 260449)


In [77]:
### Multinomial Naive Bayes Classifier ###
# The alpha parameter was determined by grid search previously
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB(alpha=0.1)
classifier.fit(x_train, y_train)

MultinomialNB(alpha=0.1)

In [78]:
y_pred = classifier.predict(x_test)

In [79]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
print("Confusion matrix\n")
print(pd.crosstab(pd.Series(y_test, name='Actual'), pd.Series(y_pred, name='Predicted')))
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted')
    recall = recall_score(y_test, y_predicted, average='weighted')
    f1 = f1_score(y_test, y_predicted, average='weighted')
    return accuracy, precision, recall, f1
accuracy, precision, recall, f1 = get_metrics(y_test, y_pred)
print("accuracy = %.3f \nprecision = %.3f \nrecall = %.3f \nf1 = %.3f" % (accuracy, precision, recall, f1))

Confusion matrix

Predicted     0     1
Actual               
0          1617   713
1           435  1783
accuracy = 0.748 
precision = 0.752 
recall = 0.748 
f1 = 0.747
